# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


# My Solution # 1 # Spark DF

In [ ]:
import findspark
findspark.init()


from pyspark import SparkContext
sc = SparkContext(appName='sga')


from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
from pyspark.sql.window import Window


se = SparkSession(sc)

In [ ]:
clickstream = se.read.csv('clickstream.csv', header=True, sep='\t')

# Сheck for errors over (and within) different sessions and users!
events = Window.partitionBy("user_id", "session_id").orderBy("timestamp")
# Creating flags for rows that either contain the 'error' substring or follow the row with the 'error' substring.
# Utilizing the window function to define the context based on users and sessions, ordered by timestamp.
clickstream_w_error_flags = (clickstream
                             .withColumn("error_flag", F.max(F.when(f.col("event_type").contains("error"), 'error').otherwise(0)).over(events)))

# Creating a new DataFrame by filtering out rows identified as errors.
# The DataFrame 'without_errors' is derived from 'clickstream_w_error_flags' and includes only rows where the 'error_flag' is not equal to 1.
# This operation effectively removes rows associated with errors, providing a dataset that excludes entries flagged as errors.
without_errors = clickstream_w_error_flags.filter(F.col('error_flag') != 'error')

# the only way to collect routes in a parallelized way is via collect_list func.
# however, we need to deduplicate cases when consecutive rows contain the same 'event_page' value
# this is done because internet -> internet is not a valid click transition and should not be considered
filtered_w_dup_flags = without_errors.withColumn("to_remove",F.when(F.lag('event_page').over(events) == F.col('event_page'), 'duplicate').otherwise(0))
no_duplicate = filtered_w_dup_flags.filter(F.col('to_remove') != 'duplicate')

# Generating a concatenated string representing the maximum route per user and session.
# The DataFrame 'no_duplicate_routes' undergoes a transformation where a new column 'sorted_route' is created.
# This column is populated by concatenating the distinct 'event_page' values within each user and session, ordered by the specified window defined by the 'events'.
# The resulting 'sorted_route' column provides a string representation of the maximum route for each unique combination of user and session.
no_duplicate_routes = no_duplicate.withColumn('sorted_route', F.concat_ws("-", F.collect_list('event_page').over(events)))

# Identifying all maximum routes for each user session.
# The DataFrame 'no_duplicate_routes' is grouped by 'user_id' and 'session_id'.
# The aggregation operation is applied to find the maximum value of the 'sorted_route' column within each group, and the resulting column is named 'route'.
# The DataFrame 'max_routes' contains information about the maximum route for every distinct user session combination.
max_routes = no_duplicate_routes\
          .groupby('user_id', 'session_id')\
          .agg(F.last('sorted_route').alias('route'))

# ANSWER
ANSWER_df = max_routes\
                .groupby('route').agg(f.count('*').alias('count'))\
                .orderBy(F.desc('count'))\
                .limit(30)

ANSWER_df.show(30)

# My Solution # 2 # Spark SQL

In [ ]:
clickstream.createOrReplaceTempView('clickstream')

ANSWER_sql = se.sql("""
WITH error_flagged AS (
    SELECT 
        user_id, 
        session_id, 
        event_page, 
        timestamp,
        MAX(CASE WHEN event_type LIKE '%error%' THEN NULL ELSE FALSE END) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS error
    FROM 
        clickstream),
dup_flagged AS (
    SELECT 
        user_id, 
        session_id,
        event_page,
        timestamp,
        CASE WHEN LAG(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) = event_page THEN NULL ELSE FALSE END AS dupl
    FROM 
        error_flagged
    WHERE
        error IS NOT NULL),
collected_routes AS (
    SELECT 
        user_id, 
        session_id,
        CONCAT_WS('-', COLLECT_LIST(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp)) AS route
    FROM 
        dup_flagged
    WHERE 
        dupl IS NOT NULL
),
routes AS (
    SELECT
        user_id, 
        session_id,
        MAX(route) AS route
    FROM 
        collected_routes
    GROUP BY
        user_id, session_id)

SELECT route, COUNT(*) as count FROM routes
GROUP BY route
ORDER BY count DESC LIMIT 30
""")

ANSWER_sql.show(30)

# My Solution # 3 # Spark RDD

In [ ]:
%%markdown
# RDD solution

In [ ]:
def remove_duplicates(row):
    return [x for i, x in enumerate(row) if i == 0 or row[i] != row[i-1]]

# Identifying rows containing errors in the 'event_type'.
errors = clickstream.rdd.filter(lambda row: 'error' in row.event_type).map(lambda row: ((row.user_id, row.session_id), row.timestamp)).reduceByKey(lambda x, y: min(x, y))

# Filtering out rows associated with errors or those occurring after an error.
no_errors = clickstream.rdd.map(lambda row: ((row.user_id, row.session_id), row)).leftOuterJoin(errors).filter(lambda row: row[1][1] is None or row[1][0].timestamp < row[1][1]).map(lambda row: row[1][0])

# Mapping rows to key-value pairs where keys are (user_id, session_id) and values are event_page.
best_routes = no_errors.map(lambda row: ((row.user_id, row.session_id), row.event_page)).groupByKey().map(lambda row: (row[0], list(row[1])))


# Applying the remove_dupl function to the list of event_pages in each group.
best_routes = best_routes.map(lambda row: (row[0], '-'.join(remove_duplicates(row[1]))))

# Creating an RDD 'ANSWERS_rdd' with counts of each unique route, sorted in descending order.
ANSWERS_rdd = best_routes.map(lambda row: (row[1], 1)).reduceByKey(lambda x, y: x + y).sortBy(lambda row: row[1], ascending=False).take(30)

# The resulting 'ANSWERS_rdd' contains the top 30 routes along with their respective counts.
ANSWERS_rdd


In [ ]:
routes_pd = ANSWER_df.toPandas()
routes_pd.to_csv('ANSWERS_df.csv', sep='\t', index=False)

routes_pd = ANSWER_sql.toPandas()
routes_pd.to_csv('ANSWERS_sql.csv', sep='\t', index=False)

with open('ANSWERS_rdd.csv', 'w') as f:
    f.write(f'route\tcount\n')
    for route, count in ANSWERS_rdd:
        f.write(f'{route}\t{count}\n')

In [ ]:
se.stop()